<a href="https://colab.research.google.com/github/Fernando232svg/esan_mlops/blob/main/Copia_de_Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow scikit-learn pandas openpyxl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.9/705.9 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import mlflow
import mlflow.sklearn


In [ ]:
def ejecutar_pipeline(path_excel, hojas):
    mlflow.set_experiment("auditoria_callcenter")


    df_base = pd.read_excel(path_excel, sheet_name=hojas[0])
    X_base = df_base[["tiempo_espera", "tono_voz", "quejas"]].astype(float)
    y_base = df_base["satisfaccion"]

    X_train, X_test, y_train, y_test = train_test_split(
        X_base, y_base, test_size=0.3, random_state=42
    )
    modelo_actual = LogisticRegression().fit(X_train, y_train)
    baseline_acc = accuracy_score(y_test, modelo_actual.predict(X_test))

    print(f"📌 Baseline ({hojas[0]}) → Accuracy: {baseline_acc:.2f}")


    for hoja in hojas[1:]:
        df_new = pd.read_excel(path_excel, sheet_name=hoja)
        X_new = df_new[["tiempo_espera", "tono_voz", "quejas"]].astype(float)
        y_new = df_new["satisfaccion"]

        X_total = pd.concat([X_base, X_new])
        y_total = pd.concat([y_base, y_new])

        X_train, X_test, y_train, y_test = train_test_split(
            X_total, y_total, test_size=0.3, random_state=42
        )
        modelo_nuevo = LogisticRegression().fit(X_train, y_train)
        new_acc = accuracy_score(y_test, modelo_nuevo.predict(X_test))

        input_example = X_new.iloc[[0]]

        with mlflow.start_run():
            mlflow.log_param("dataset", hoja)
            mlflow.log_metric("baseline_accuracy", baseline_acc)
            mlflow.log_metric("new_accuracy", new_acc)

            if new_acc > baseline_acc:
                mlflow.sklearn.log_model(
                    sk_model=modelo_nuevo,
                    artifact_path="modelo_publicado",
                    input_example=input_example
                )
                decision = f"✅ Nuevo modelo publicado con {hoja}"
                modelo_actual = modelo_nuevo
                baseline_acc = new_acc
                X_base, y_base = X_total, y_total
            else:
                mlflow.sklearn.log_model(
                    sk_model=modelo_actual,
                    artifact_path="modelo_mantenido",
                    input_example=input_example
                )
                decision = f"❌ Se mantiene el modelo anterior (no mejora con {hoja})"

        print(f"➡️ {hoja}: accuracy={new_acc:.2f} → {decision}")


In [ ]:
from google.colab import files


uploaded = files.upload()
path_excel = list(uploaded.keys())[0]

xls = pd.ExcelFile(path_excel)
hojas = xls.sheet_names
print("✅ Hojas detectadas:", hojas)

ejecutar_pipeline(path_excel, hojas)


Saving datasets_callcenter_5meses_ruidoso.xlsx to datasets_callcenter_5meses_ruidoso.xlsx


2025/09/14 23:35:01 INFO mlflow.tracking.fluent: Experiment with name 'auditoria_callcenter' does not exist. Creating a new experiment.


✅ Hojas detectadas: ['dataset_malo', 'dataset_mejorado', 'dataset_mes3', 'dataset_mes4', 'dataset_mes5']
📌 Baseline (dataset_malo) → Accuracy: 0.00


2025/09/14 23:35:01 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/14 23:35:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


➡️ dataset_mejorado: accuracy=0.33 → ✅ Nuevo modelo publicado con dataset_mejorado


...

➡️ dataset_mes3: accuracy=0.56 → ✅ Nuevo modelo publicado con dataset_mes3


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/14 23:35:14 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


➡️ dataset_mes4: accuracy=0.67 → ✅ Nuevo modelo publicado con dataset_mes4


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


➡️ dataset_mes5: accuracy=0.67 → ❌ Se mantiene el modelo anterior (no mejora con dataset_mes5)


In [ ]:
from google.colab import files

uploaded = files.upload()
path_excel = list(uploaded.keys())[0]

xls = pd.ExcelFile(path_excel)
hojas = xls.sheet_names
print("✅ Hojas detectadas:", hojas)

ejecutar_pipeline(path_excel, hojas)


2025/09/14 23:51:35 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Saving datasets_callcenter_6meses.xlsx to datasets_callcenter_6meses.xlsx
✅ Hojas detectadas: ['dataset_malo', 'dataset_mejorado', 'dataset_mes3', 'dataset_mes4', 'dataset_mes5', 'dataset_mes6']
📌 Baseline (dataset_malo) → Accuracy: 0.00


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/14 23:51:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


➡️ dataset_mejorado: accuracy=0.33 → ✅ Nuevo modelo publicado con dataset_mejorado


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/14 23:51:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

➡️ dataset_mes4: accuracy=0.67 → ✅ Nuevo modelo publicado con dataset_mes4


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/09/14 23:51:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


➡️ dataset_mes5: accuracy=0.67 → ❌ Se mantiene el modelo anterior (no mejora con dataset_mes5)


/usr/local/lib/python3.12/dist-packages/mlflow/types/utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


➡️ dataset_mes6: accuracy=0.80 → ✅ Nuevo modelo publicado con dataset_mes6
